# Introduction to Ray Data: Ray Data + Structured Data
© 2025, Anyscale. All Rights Reserved

💻 **Launch Locally**: You can run this notebook locally, but performance will be reduced.

🚀 **Launch on Cloud**: A Ray Cluster (Click [here](http://console.anyscale.com/register) to easily start a Ray cluster on Anyscale) is recommended to run this notebook.

This notebook will provide an overview of Ray Data and how to use it to load, and transform data in a distributed manner.

<div class="alert alert-block alert-info">
<b> Here is the roadmap for this notebook:</b>
<ul>
    <li><b>Part 0:</b> What is Ray Data?</a></li>
    <li><b>Part 1:</b> How to use Ray Data?</a></li>
    <li><b>Part 2:</b> Loading Data</a></li>
    <li><b>Part 3:</b> Transforming Data</a></li>
    <li><b>Part 4:</b> Writing Data</a></li>
    <li><b>Part 5:</b> Data Operations: Shuffling, Grouping and Aggregation</a></li>
    <li><b>Part 6:</b> When to use Ray Data</a></li>
    <li><b>Part 7:</b> Ray Data in Production</a></li>
    <li><b>Part 8:</b> Upcoming Features in Ray Data</a></li>
</ul>
</div>

**Imports**

In [1]:
import ray
import pandas as pd

## 0. What is Ray Data?

Ray Data is a distributed data processing library that provides a Python API for parallel data processing. 

It is built on top of Ray, a fast and simple framework for building and running distributed applications. Ray Data is designed to be easy to use, scalable, and fault-tolerant.

### 1. How to Use Ray Data?

You typically should use the Ray Data API in this way:

1. **Create a Ray Dataset** from external storage or in-memory data.
2. **Apply transformations** to the data.
3. **Write the outputs** to external storage or **feed the outputs** to training workers.


## 2. Loading Data

Our Dataset is the New York City Taxi & Limousine Commission's Trip Record Data

**Dataset features**

| Column | Description | 
| ------ | ----------- |
| `trip_distance` | Float representing trip distance in miles. |
| `passenger_count` | The number of passengers |
| `PULocationID` | TLC Taxi Zone in which the taximeter was engaged | 
| `DOLocationID` | TLC Taxi Zone in which the taximeter was disengaged | 
| `payment_type` | A numeric code signifying how the passenger paid for the trip. |
| `tolls_amount` | Total amount of all tolls paid in trip. | 
| `tip_amount` | Tip amount – This field is automatically populated for credit card tips. Cash tips are not included. | 
| `total_amount` | The total amount charged to passengers. Does not include cash tips. |


In [2]:
COLUMNS = [
    "trip_distance",
    "passenger_count",
    "PULocationID",
    "DOLocationID",
    "payment_type",
    "tolls_amount",
    "tip_amount",
    "total_amount",
]

DATA_PATH = "s3://anyscale-public-materials/nyc-taxi-cab"

Let's read the data for a single month. It takes up to 2 minutes to run.

In [3]:
df = pd.read_parquet(
    f"{DATA_PATH}/yellow_tripdata_2011-05.parquet",
    columns=COLUMNS,
)

df.head()

,trip_distance,passenger_count,PULocationID,DOLocationID,payment_type,tolls_amount,tip_amount,total_amount
0,1.48,1,148,231,2,0.0,0.0,8.7
1,1.63,1,132,132,1,0.0,2.0,11.5
2,1.10,2,79,107,1,0.0,1.0,8.1
3,2.80,1,234,141,1,0.0,1.5,12.2
4,1.70,1,237,140,1,0.0,1.0,8.9


Let's check how much memory the dataset is using.

In [6]:
total_memory_usage = df.memory_usage(deep=True).sum().sum() / 1024**2 # result will be in MB
print(f"Total memory usage: {total_memory_usage:.2f} MB")

Total memory usage: 949.39 MB


Let's check how many files there are in the dataset

In [9]:
# !aws s3 ls s3://anyscale-public-materials/nyc-taxi-cab/ --human-readable | wc -l
# 156 files (it won't work without installing the aws cli)
print(f"Number of files: {156}")

Number of files: 156


We are not making use of all the columns and are already consuming ~1GB of data per file -> will quickly become a problem if you want to scale to entire dataset (~155 files) if we are running on a small node.

Let's instead make use of a distributed data preprocessing library like Ray Data to load the full dataset in a distributed manner.

In [4]:
ds = ray.data.read_parquet(
    DATA_PATH,
    columns=COLUMNS,
)

Metadata Fetch Progress 0:   0%|          | 0.00/25.0 [00:00<?, ? task/s]

2025-12-17 00:11:53,306	INFO worker.py:1879 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Parquet Files Sample 0:   0%|          | 0.00/2.00 [00:00<?, ? file/s]

There are Ray data equivalents for common pandas functions like `read_csv`, `read_parquet`, `read_json`, etc.

Refer to the [Input/Output docs](https://docs.ray.io/en/latest/data/api/input_output.html) for a comprehensive list of read functions.

### Dataset

Let's view our dataset

In [5]:
ds

Dataset(
   num_rows=1340417181,
   schema={
      trip_distance: double,
      passenger_count: int64,
      PULocationID: int64,
      DOLocationID: int64,
      payment_type: int64,
      tolls_amount: double,
      tip_amount: double,
      total_amount: double
   }
)

Ray Data by default adopts **lazy execution** this means that the data is not loaded into memory until it is needed. Instead only a small part of the dataset is loaded into memory to infer the schema. (a.k.a execute "ds" is equivalente to pd.DataFrame.head(), or PySpark Limite)

A Dataset specifies a sequence of transformations that will be applied to the data. 

The data itself will be organized into blocks, where each block is a collection of rows.

The following figure visualizes a tabular dataset with three blocks, each block holding 1000 rows each:

<img src='https://docs.ray.io/en/releases-2.6.1/_images/dataset-arch.svg' width=50%/>

Since a Dataset is just a list of Ray object references, it can be freely passed between Ray tasks, actors, and libraries like any other object reference. This flexibility is a unique characteristic of Ray Datasets.

## 3. Transforming Data

Let's create a simple function to generate features from the data. Here is how we would do so using pandas

In [6]:
def adjust_total_amount(df: pd.DataFrame) -> pd.DataFrame:
    df["adjusted_total_amount"] = df["total_amount"] - df["tip_amount"]
    return df

df = adjust_total_amount(df)

We can take the same function and apply it to the Ray dataset using `map_batches`. 

`map_batches` will batch each block of the dataset and apply the function to each batch in parallel.

In [7]:
ds_adjusted = ds.map_batches(adjust_total_amount, batch_format="pandas")

<div class="alert alert-block alert-warning">
<b>Note</b> 

The default `batch_format` in Ray Data is `numpy`, which means that the data is returned as a numpy array. For optimal performance, it is recommended to **avoid converting the data to pandas dataframes unless necessary**.

</div>

Let's add another transformation, for the sake of this example, we will add a simple transformation to calculate the tip percentage.

In [9]:
def compute_tip_percentage(df: pd.DataFrame) -> pd.DataFrame:
    df["tip_percentage"] = df["tip_amount"] / df["total_amount"]
    return df

df = compute_tip_percentage(df)

We would apply it again using `map_batches`. Note that we can control certain additional parameters such as the batch size to use. (how does batch_size relates to the number of rows by each ObjectRef? Does it make some kind of repartition?)

In [10]:
# TODO: how does batch_size relates to the number of rows by each ObjectRef? Does it make some kind of repartition?
ds_tip = ds_adjusted.map_batches(compute_tip_percentage, batch_format="pandas", batch_size=1024)

### Execution mode

Most transformations are **lazy** in Ray Data - i.e. they don't execute until you either:
- **write a dataset to storage**
- explicitly **materialize** the data
- **iterate over the dataset** (usually when feeding data to model training).

To explicitly *materialize* a very small subset of the data, you can use the `take_batch` method.

In [20]:
ds.take_batch()

2025-12-17 00:31:11,900	INFO logging.py:290 -- Registered dataset logger for dataset dataset_10_0
2025-12-17 00:31:11,906	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_10_0. Full logs are in /tmp/ray/session_2025-12-17_00-11-49_070128_44566/logs/ray-data
2025-12-17 00:31:11,907	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_10_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> LimitOperator[limit=20]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(25) 1: 0.00 row [00:00, ? row/s]

- limit=20 2: 0.00 row [00:00, ? row/s]

2025-12-17 00:31:33,744	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_10_0 execution finished in 21.84 seconds


{'trip_distance': array([ 1.48,  1.63,  1.1 ,  2.8 ,  1.7 ,  0.5 ,  1.4 ,  0.8 ,  2.1 ,
         1.3 ,  2.6 ,  0.8 ,  1.9 ,  0.7 ,  1.3 ,  1.6 ,  2.1 ,  0.8 ,
        19.2 ,  0.  ]),
 'passenger_count': array([1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2]),
 'PULocationID': array([148, 132,  79, 234, 237, 237, 161, 186,  90, 229,  48, 234, 170,
         79, 107, 114, 231, 234, 132, 162]),
 'DOLocationID': array([231, 132, 107, 141, 140, 237, 234,  90, 161,  48, 107, 170,  79,
        107, 114, 231, 234, 137, 263, 164]),
 'payment_type': array([2, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2]),
 'tolls_amount': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 0. , 4.8, 0. ]),
 'tip_amount': array([ 0.  ,  2.  ,  1.  ,  1.5 ,  1.  ,  1.  ,  0.  ,  1.02,  1.82,
         0.  ,  1.75,  0.  ,  2.73,  1.7 ,  0.  ,  1.  ,  0.  ,  0.  ,
        10.06,  0.  ]),
 'total_amount': array([ 8.7 , 11.5 ,  8.1 , 12.2 ,  8.9 ,  

Let's view a batch of the transformed data.

In [21]:
ds_tip.take_batch()

2025-12-17 00:31:33,764	INFO logging.py:290 -- Registered dataset logger for dataset dataset_11_0
2025-12-17 00:31:33,772	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_11_0. Full logs are in /tmp/ray/session_2025-12-17_00-11-49_070128_44566/logs/ray-data
2025-12-17 00:31:33,772	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_11_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> TaskPoolMapOperator[MapBatches(adjust_total_amount)->MapBatches(compute_tip_percentage)] -> LimitOperator[limit=20]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet 1: 0.00 row [00:00, ? row/s]

- MapBatches(adjust_total_amount)->MapBatches(compute_tip_percentage) 2: 0.00 row [00:00, ? row/s]

- limit=20 3: 0.00 row [00:00, ? row/s]

2025-12-17 00:31:50,633	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_11_0 execution finished in 16.86 seconds


{'trip_distance': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 8. , 1.6, 2.5, 3.9, 0.6, 0.9,
        3.9, 0.7, 1.9, 3.3, 5.3, 1. , 4.3]),
 'passenger_count': array([4, 4, 4, 5, 5, 1, 1, 1, 1, 4, 2, 2, 4, 1, 2, 2, 2, 2, 1, 1]),
 'PULocationID': array([145, 264, 264, 146, 146, 146, 146, 138, 170, 237, 170,  90,  90,
         90, 113,  79, 170, 142, 114, 249]),
 'DOLocationID': array([145, 264, 264, 146, 146, 146, 146, 256, 237, 170, 239,  90, 186,
        238,  79, 170, 142, 112, 249, 143]),
 'payment_type': array([1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2]),
 'tolls_amount': array([0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. , 0. , 0. , 4.8, 0. , 0. ]),
 'tip_amount': array([0.28, 0.24, 1.11, 0.  , 0.11, 0.  , 0.  , 0.  , 0.  , 0.  , 2.38,
        0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  ]),
 'total_amount': array([ 4.18,  6.94,  5.01,  3.9 ,  3.61,  3.5 ,  3.5 , 21.1 , 10.3 ,
         9.1 , 18.28,  5.1 ,  7.5 , 16.3 ,  4.6 ,

## 4. Writing Data

Let's write the adjusted data. Here is how we would do it with pandas:

In [22]:
import os

working_dir = os.getcwd()
# storage_folder = '/mnt/cluster_storage' # Modify this path to your local folder if it runs on your local environment
storage_folder = working_dir

In [21]:
df.to_parquet(f"{storage_folder}/adjusted_data.parquet")

Let's check the file we just wrote:

In [22]:
!ls -lh {storage_folder}/adjusted_data.parquet

-rw-r--r--@ 1 rbrosa  staff   140M Dec 16 00:51 /Users/rbrosa/Documents/github_personal/intro-to-ray/notebooks/ray_data/adjusted_data.parquet


Here is how we would do so with Ray Data:

In [23]:
!rm -rf {storage_folder}/adjusted_data_ray/ # let's remove the directory if it exists
ds_limited = ds_adjusted.limit(df.shape[0]) # we limit to avoid writing too much data
ds_limited.write_parquet(f"{storage_folder}/adjusted_data_ray/")

2025-12-17 00:32:06,032	INFO logging.py:290 -- Registered dataset logger for dataset dataset_14_0
2025-12-17 00:32:06,037	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_14_0. Full logs are in /tmp/ray/session_2025-12-17_00-11-49_070128_44566/logs/ray-data
2025-12-17 00:32:06,038	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_14_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet->MapBatches(adjust_total_amount)] -> LimitOperator[limit=15554868] -> TaskPoolMapOperator[Write]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->MapBatches(adjust_total_amount) 1: 0.00 row [00:00, ? row/s]

- limit=15554868 2: 0.00 row [00:00, ? row/s]

- Write 3: 0.00 row [00:00, ? row/s]

2025-12-17 00:32:53,631	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_14_0 execution finished in 47.59 seconds
2025-12-17 00:32:53,666	INFO dataset.py:4537 -- Data sink Parquet finished. 15554868 rows and 1.0GB data written.


There are Ray data equivalents for common pandas functions like `write_parquet` for `to_parquet`, `write_csv` for `to_csv`, etc.

See the [Input/Output docs](https://docs.ray.io/en/latest/data/api/input_output.html) for a comprehensive list of write functions.

Let's check the files in the directory:

In [24]:
!ls -lh {storage_folder}/adjusted_data_ray/

total 245488
-rw-r--r--@ 1 rbrosa  staff    14M Dec 16 00:52 7_000000_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    15M Dec 16 00:52 7_000001_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    14M Dec 16 00:52 7_000002_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    14M Dec 16 00:52 7_000003_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    15M Dec 16 00:52 7_000004_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    14M Dec 16 00:52 7_000005_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    15M Dec 16 00:52 7_000006_000000.parquet
-rw-r--r--@ 1 rbrosa  staff   2.9M Dec 16 00:52 7_000007_000000.parquet
-rw-r--r--@ 1 rbrosa  staff    14M Dec 16 00:52 7_000008_000000.parquet
-rw-r--r--@ 1 rbrosa  staff   1.7M Dec 16 00:52 7_000009_000000.parquet


Notice that we have **multiple files** in the directory. This is because Ray Data writes data in a **distributed manner**. 

**Each task writes its own file**, and the number of files is proportional to the number of CPUs in the cluster.

**Ray Data uses Ray tasks** to process data.

When reading from a file-based datasource (e.g., S3, GCS). Each read task reads its assigned files and produces an output block which in turn is consumed by the next task in the pipeline.
    
<img src="https://anyscale-public-materials.s3.us-west-2.amazonaws.com/ray-summit/rag-app/dataset-read-cropped-v2.svg" width="500px">

<div class="alert alert-block alert-warning">
<b>Note</b> 

We passed `/mnt/cluster_storage/` as the path to write the data. This is a path on the Ray cluster's shared storage. If instead you use a path that is only local to one of the nodes in a multi-node cluster, you will see errors like `FileNotFoundError: [Errno 2] No such file or directory: '/path/to/file'`.

This is because Ray Data is designed to work with distributed storage systems like S3, HDFS, etc. If you want to write to local storage, you can add a special prefix `local://` to the path. For example, `local:///path/to/file`. However to do so you will need to ensure that Ray is enabled to schedule and run tasks on the head node of the cluster.
</div>

## 5. Data Operations: Shuffling, Grouping and Aggregation

Let's look at some more involved transformations.

### Shuffling data 

There are different options to shuffle data in Ray Data of varying degrees of randomness and performance.

#### File based shuffle on read

To randomly shuffle the ordering of input files before reading, use the shuffle="files" parameter.

In [13]:
ds_file_shuffled = ray.data.read_parquet(DATA_PATH, columns=COLUMNS, shuffle="files")

Metadata Fetch Progress 0:   0%|          | 0.00/25.0 [00:00<?, ? task/s]

Parquet Files Sample 0:   0%|          | 0.00/2.00 [00:00<?, ? file/s]

In [14]:
ds_file_shuffled

Dataset(
   num_rows=1340417181,
   schema={
      trip_distance: double,
      passenger_count: int64,
      PULocationID: int64,
      DOLocationID: int64,
      payment_type: int64,
      tolls_amount: double,
      tip_amount: double,
      total_amount: double
   }
)

#### Shuffling block order
This option randomizes the order of blocks in a dataset.

Applying this operation alone doesn’t involve heavy computation and communication. However, it requires Ray Data to materialize all blocks before applying the operation.

Let's read the data and shuffle the block order.

In [15]:
ds = (
    ray.data.read_parquet(
        "s3://anyscale-public-materials/nyc-taxi-cab/yellow_tripdata_2011-05.parquet",
        columns=COLUMNS,
    )
)

Parquet Files Sample 0:   0%|          | 0.00/1.00 [00:00<?, ? file/s]

To perform block order shuffling, use `randomize_block_order`.

In [16]:
ds_block_based_shuffle = ds.randomize_block_order()
ds_block_based_shuffle.to_pandas()

2025-12-17 00:26:54,300	INFO logging.py:290 -- Registered dataset logger for dataset dataset_7_0
2025-12-17 00:26:54,310	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_7_0. Full logs are in /tmp/ray/session_2025-12-17_00-11-49_070128_44566/logs/ray-data
2025-12-17 00:26:54,317	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_7_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> AllToAllOperator[RandomizeBlockOrder]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(25) 1: 0.00 row [00:00, ? row/s]

- RandomizeBlockOrder 2: 0.00 row [00:00, ? row/s]

2025-12-17 00:27:19,725	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_7_0 execution finished in 25.42 seconds


,trip_distance,passenger_count,PULocationID,DOLocationID,payment_type,tolls_amount,tip_amount,total_amount
0,1.70,1,230,90,2,0.0,0.00,6.60
1,3.10,1,90,162,2,0.0,0.00,10.20
2,2.90,1,230,141,2,0.0,0.00,9.40
3,12.07,1,132,49,2,0.0,0.00,32.60
4,0.00,1,79,114,1,0.0,1.15,6.95
...,...,...,...,...,...,...,...,...
15554863,2.60,1,150,265,1,0.0,2.00,11.10
15554864,1.41,1,265,265,2,0.0,0.00,7.90
15554865,4.67,1,265,265,1,0.0,0.50,14.40
15554866,0.96,1,265,265,2,0.0,0.00,5.90


#### Shuffle all rows globally
To randomly shuffle all rows globally, call `random_shuffle()`. This is the slowest option for shuffle, and requires transferring data across network between workers. This option achieves the best randomness among all options.


In [ ]:
ds_row_based_shuffle = ds.random_shuffle()

In [ ]:
ds_row_based_shuffle.to_pandas()

#### Custom batching using `groupby` and aggregations

In case you want to generate batches according to a specific key, you can use `groupby` to group the data by the key and then use `map_groups` to apply the transformation.

For instance, let's compute the average trip distance per passenger count. Here is how we would do it with pandas:

In [17]:
df.groupby("payment_type")["trip_distance"].mean()

payment_type
1    3.192318
2    2.592786
Name: trip_distance, dtype: float64

Here is how we would do the same operation with Ray Data:

In [18]:
num_cpus = 8
ds.repartition(num_cpus).groupby("payment_type").mean("trip_distance").to_pandas()

2025-12-17 00:28:50,717	INFO logging.py:290 -- Registered dataset logger for dataset dataset_9_0
2025-12-17 00:28:50,726	INFO streaming_executor.py:117 -- Starting execution of Dataset dataset_9_0. Full logs are in /tmp/ray/session_2025-12-17_00-11-49_070128_44566/logs/ray-data
2025-12-17 00:28:50,726	INFO streaming_executor.py:118 -- Execution plan of Dataset dataset_9_0: InputDataBuffer[Input] -> TaskPoolMapOperator[ReadParquet] -> AllToAllOperator[Repartition] -> AllToAllOperator[Aggregate]


Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet->SplitBlocks(200) 1: 0.00 row [00:00, ? row/s]

- Repartition 2: 0.00 row [00:00, ? row/s]

Split Repartition 3:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

...

Sort Sample 5:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Map 6:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

Shuffle Reduce 7:   0%|          | 0.00/1.00 [00:00<?, ? row/s]

2025-12-17 00:29:26,034	INFO streaming_executor.py:220 -- ✔️  Dataset dataset_9_0 execution finished in 35.31 seconds


,payment_type,mean(trip_distance)
0,1,3.192318
1,2,2.592786


Here are the main aggregation functions available in Ray Data:
- count
- max
- mean
- min
- sum
- std

See [relevant docs page here](https://docs.ray.io/en/latest/data/api/grouped_data.html#computations-or-descriptive-stats)

<div class="alert alert-block alert-warning">

<b>Note:</b> This is an area of active development in Ray Data. The current implementation of groupby is not as optimized as it could be. We are working on improving the performance of `groupby` and `map_groups` operations.

For more details, the current implementation makes use of a sort operation which instead can be done using a hash-based implementation. Additionally, we had to repartition the data to maximize parallelism - in the future Ray Data should be able to dynamically repartition the data to maximize parallelism.

</div>

## 6. When to use Ray Data

Ray Data is especially performant when needing to:
- run data processing in a **streaming fashion** 
- run across a **large dataset**
- run inside a **heterogeneous cluster of CPUs and GPUs**.

Here is one use case for Batch Inference with Ray Data over a large dataset:

<img src='https://docs.ray.io/en/releases-2.6.1/_images/stream-example.png' width=60%/>


Ray Data also integrates seamlessly with Ray Train, making it an optimal choice for **data preprocessing in machine learning training pipelines**. Especially when you need to:
- **Independently scale out data loading and transformation** from model training.
- **Enable fault tolerance** for model training.


### 7. Ray Data in Production

1. Runway AI is using Ray Data to scale its ML workloads. See [this interview with Runway AI](https://siliconangle.com/2024/10/02/runway-transforming-ai-driven-filmmaking-innovative-tools-techniques-raysummit/) to learn more.
2. Netflix is using Ray Data for multi-modal batch inference pipelines. See [this talk at the Ray Summit 2024](https://raysummit.anyscale.com/flow/anyscale/raysummit2024/landing/page/sessioncatalog/session/1722028596844001bCg0) to learn more.
3. Spotify uses Ray Data for large-scale data processing. See [this talk at the Ray Summit 2023](https://www.anyscale.com/blog/how-spotify-built-a-robust-ray-platform-with-a-frictionless-developer) to learn more.

### 8. Upcoming Features in Ray Data

Here are some relevant upcoming features in Ray Data:

For structured data:
- improved `groupby` and `map_groups` performance
- using parquet metadata for computing statistics like `count`
- enabling predicate pushdown for parquet files when calling `filter`
- supporting `join` and `merge` operations
- optimizing performance of the `Preprocessor` API for distributed feature engineering
- running spark on Ray more seamlessly


For all data types:
- data checkpointing for fault tolerance
- optimizing data connectors
- concurrent execution of multiple datasets

In [24]:
# Run this cell for file cleanup 
!rm {storage_folder}/adjusted_data.parquet
!rm -rf {storage_folder}/adjusted_data_ray/